# Практическая работа №2 «Идентификация и аутентификация (RSA, схемы Шнорра и Фейге-Фиата-Шамира)»
В практической работе необходимо привести последовательность выполнения процедур идентификации/аутентификации с использованием следующих способов:
- на основе алгоритма RSA;
- по схеме Шнорра;
- по схеме Фейге-Фиата-Шамира.  

При оформлении отчета необходимо привести таблицы генерации ключей и аутентификации. В качестве случайного числа (k или r) принять коды, соответственно, 1-ой, 2-ой и 3-ей буквы своей фамилии согласно их положению в алфавите. Отчет прикрепляется в edu.susu.ru


In [1]:
import numpy as np
import json

In [2]:
def generate_primes(max_prime):  
    primes = []
    for possible_prime in range(2, max_prime + 1):
        is_prime = True
        for num in range(2, int(possible_prime ** 0.5) + 1):
            if possible_prime % num == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(possible_prime)
    return(primes)

In [3]:
def gcd_ext(a, b):
    """
    Find a*x+b*y = gcd(a, b)
    """
    if a == 0:
        return b, 0, 1
    d, x, y = gcd_ext(b % a, a)
    return d, y - (b // a) * x, x

In [4]:
def rev_element_by_modulo(num, mod):
    g, x, y = gcd_ext(num, mod)
    if g != 1:
        return None
    return (x % mod + mod) % mod

In [5]:
def power_with_modulo(a, n, m):
    res = 1
    while n > 0:
        if (n & 1) == 1:
            res = (res * a) % m
        a = (a * a) % m
        n >>= 1
    return res

In [6]:
with open('instance/settings.json') as f:
    settings = json.load(f)
    LAST_NAME = settings['last_name']

In [7]:
random = np.random.RandomState(seed=42)
primes = generate_primes(30)

## Алгоритм RSA

### Генерация ключей

**A** генерирует ключи

Выбирается простые $p$ и $q$

In [8]:
p, q = random.choice(primes, 2)
md = f'$p = {p}, q = {q}$'

{{md}}

Вычисляется произведение $n = p * q$

In [9]:
n = p * q
md = f'$n = {n}$'

{{md}}

Вычисляется функция Эйлера $\phi(n)$

In [10]:
phi_n = (p - 1) * (q - 1)
md = f'$\phi(n) = \phi(p)*\phi(q) = (p - 1) * (q - 1) = {phi_n}$'

{{md}}

Выбирается открытый ключ $e$, как произвольное число $(0 < e < n)$, взаимно простое с результатом функции Эйлера.

In [11]:
numbers = np.arange(1, n)
random.shuffle(numbers)
e = next(num for num in numbers if np.gcd(num, phi_n) == 1)
md = f'$e = {e}$'

{{md}}

Вычисляется закрытый ключ $d$, как обратное число к $e$ по модулю $\phi(n)$, из соотношения $(d*e)$ mod $\phi(n) = 1$.

In [12]:
d = rev_element_by_modulo(e, phi_n)
md = 'Обратного числа нет к $e$ по данному модулю нет' if d is None else f'$d = {d}$' 

{{md}}

Публикуются открытый ключ $(e, n)$

### Ключи

In [13]:
public_key = (e, n)
private_key = d

In [14]:
md0 = f'**public key:** $(e, n) = {public_key}$'
md1 = f'**private key:** $d = {private_key}$'

{{md0}}  
{{md1}} 

### Аутентификация

> __Замечание__  
Используем бинарное возведение в степень по модулю
- Избегаем переполнение
- Быстро возводим в степень

 **Б** выбирает случайное число $k \in \{1, …, n-1\}$, вычисляет $r = k^e$ mod $n$ и посылает $r$ __A__.

Выбираем $k$ как сумму позиций первых трех букв в алфавите по модулю $n - 1$ и смещением в $1$

In [15]:
k =  sum(map(lambda x: ord(x) - ord('a'), LAST_NAME[:3])) % (n - 1) + 1
r = power_with_modulo(k, e, n)
md = f'$k = {k}, r = {r}$'

{{md}}

**А** вычисляет $k^\prime = r^d$ mod $n$ и посылает $k^\prime$ __Б__.

In [16]:
k_prime = power_with_modulo(r, d, n)
md = f'$k^\\prime = {k_prime}$'

{{ md }}

**Б** проверяет соотношение $k = k^\prime$ и, если оно истинно, принимает доказательство, в противном случае - отвергает.

In [17]:
approved = k == k_prime
md = f'{"Подтверждено" if approved else "Не подтверждено"}'

{{md}}

## Схема Клауса Шнорра

### Генерация

__A__ генерирует ключи

Выбираются два простых числа $p$ и $q$ такие, что $(p - 1)$ mod $q = 0$.

In [18]:
p = 23
q = 11

Выбирается секретный ключ $x \in \{1, …, q-1\}$.

In [19]:
x = 8

Выбирается $g$ такое, что $g^q$ mod $p = 1$.

In [20]:
g = 3 

Вычисляется открытый ключ $y$ такой, что $(g^x * y)$ mod $p = 1$.

In [21]:
y = 4

Публикация открытого ключа $y$.

### Ключи

In [22]:
public_key = (y, g, p)
private_key = (x, q)

In [23]:
md0 = f'**public key:** $(y, g, p) = {public_key}$'
md1 = f'**private key:** $(x, q) = {private_key}$'

{{md0}}  
{{md1}}

### Аутентификация

**А** выбирает случайное число $k \in \{1, …, q-1\}$, вычисляет $r = g^k$ mod $p$ и посылает $r$ __Б__.

In [24]:
k = sum(map(lambda x: ord(x) - ord('a'), LAST_NAME[:3])) % (q - 1) + 1
r = power_with_modulo(g, k, p)
md = f'$k = {k}, r = {r}$'

{{md}}

__Б__ выбирает случайное число $e \in \{0, …, 2^t-1\}$, где $t$ - некоторый параметр, и посылает $e$ **А**.

In [25]:
e = random.choice(np.arange(0, 1 << 20))
md = f'$e = {e}$'

{{md}}

**А** вычисляет $s = (k + x * e)$ mod $q$ и посылает $s$ __Б__.

In [26]:
s = (k + x * e) % q
md = f'$s = {s}$'

{{md}}

**Б** проверяет соотношение $r = (g^s * y^e)$ mod $p$ и, если оно выполняется, принимает доказательство, в противном случае - отвергает.

In [27]:
check = (power_with_modulo(g, s, p) * power_with_modulo(y, e, p)) % p
approved = r == check
md = f'{"Подтверждено" if approved else "Не подтверждено"}'

{{md}}